In [8]:
from collections import defaultdict
import json

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib import rcParams
import matplotlib.cm as cm
import matplotlib as mpl

#colorbrewer2 Dark2 qualitative color table
dark2_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843)]

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 150
rcParams['axes.color_cycle'] = dark2_colors
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = 'white'
rcParams['font.size'] = 14
rcParams['patch.edgecolor'] = 'white'
rcParams['patch.facecolor'] = dark2_colors[0]
rcParams['font.family'] = 'StixGeneral'


def remove_border(axes=None, top=False, right=False, left=True, bottom=True):
    """
    Minimize chartjunk by stripping out unnecesasry plot borders and axis ticks
    
    The top/right/left/bottom keywords toggle whether the corresponding plot border is drawn
    """
    ax = axes or plt.gca()
    ax.spines['top'].set_visible(top)
    ax.spines['right'].set_visible(right)
    ax.spines['left'].set_visible(left)
    ax.spines['bottom'].set_visible(bottom)
    
    #turn off all ticks
    ax.yaxis.set_ticks_position('none')
    ax.xaxis.set_ticks_position('none')
    
    #now re-enable visibles
    if top:
        ax.xaxis.tick_top()
    if bottom:
        ax.xaxis.tick_bottom()
    if left:
        ax.yaxis.tick_left()
    if right:
        ax.yaxis.tick_right()
        
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

C:\Users\Administrator\Anaconda3\lib\site-packages\matplotlib\__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [9]:
import pandas as pd
df=pd.read_csv("data/dataset.csv")
print(len(df.columns))
print(len(df.index))

10
10


In [10]:
print(df.columns)

Index(['year', 'employment_ratio', 'household_expenditure', 'export_value_index', 'energy_use', 'net_portfolio_investment', 'import_value_index', 'population', 'GDP', 'SP500'], dtype='object')


In [11]:
df.head()

,year,employment_ratio,household_expenditure,export_value_index,energy_use,net_portfolio_investment,import_value_index,population,GDP,SP500
0,2006,63.119999,9.980000e+12,131.211585,7697.652535,-6.330000e+11,152.312952,298379912,1.480000e+13,1318.309153
1,2007,62.990002,1.020000e+13,146.843915,7758.165986,-7.760000e+11,160.438577,301231207,1.510000e+13,1478.095825
2,2008,62.180000,1.020000e+13,164.651792,7488.081921,-8.080000e+11,172.277225,304093966,1.500000e+13,1215.221659
3,2009,59.320000,1.000000e+13,135.058024,7056.783653,1.853000e+10,127.475240,306771529,1.460000e+13,948.517492
4,2010,58.470001,1.020000e+13,163.507554,7160.939243,-6.210000e+11,156.371309,309348193,1.500000e+13,1130.681661


In [12]:
smaller_frame=df[['GDP', 'SP500', 'employment_ratio']]
from pandas.plotting import scatter_matrix
axeslist=scatter_matrix(smaller_frame, alpha=0.8, figsize=(12, 12), diagonal="kde")
for ax in axeslist.flatten():
    ax.grid(False)

In [13]:
smaller_frame.corr()

,GDP,SP500,employment_ratio
GDP,1.000000,0.948470,-0.395246
SP500,0.948470,1.000000,-0.132357
employment_ratio,-0.395246,-0.132357,1.000000


In [14]:
from sklearn.linear_model import LinearRegression
X_HD=smaller_frame[['GDP', 'SP500']].values
X_HDn=(X_HD - X_HD.mean(axis=0))/X_HD.std(axis=0)
radius_std_vec=X_HDn[:,0]
radius_std=radius_std_vec.reshape(-1,1)
texture_std_vec=X_HDn[:,1]
texture_std=texture_std_vec.reshape(-1,1)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(radius_std, texture_std_vec)

In [16]:
clf1 = LinearRegression()
clf1.fit(X_train, y_train)
predicted_train = clf1.predict(X_train)
predicted_test = clf1.predict(X_test)
trains=X_train.reshape(1,-1).flatten()
tests=X_test.reshape(1,-1).flatten()
print (clf1.coef_, clf1.intercept_)

[ 0.95576303] -0.0277235537664


In [17]:
plt.scatter(educ_coll_std_vec, average_income_std_vec,c='r')
plt.plot(trains, predicted_train, c='g', alpha=0.5)
plt.plot(tests, predicted_test, c='g', alpha=0.2)

NameError: name 'educ_coll_std_vec' is not defined